In [2]:
import numpy as np

In [3]:
import jax

In [3]:
# import os
# import ctypes
# import sys


# modules_to_clean = []
# for name in list(sys.modules.keys()):
#     if any(substring in name for substring in ['h5py', 'prometheus', '_hdf5', '_h5py', '_errors']):
#         modules_to_clean.append(name)

# for mod in modules_to_clean:
#     if mod in sys.modules:
#         del sys.modules[mod]

# intel_paths = [
#     "/software/local/intel/parallel_studio_xe_2020_upd4/compilers_and_libraries_2020.4.304/linux/compiler/lib/intel64",
#     "/software/local/intel/parallel_studio_xe_2020_upd4/compilers_and_libraries_2020.4.304/linux/mkl/lib/intel64",
# ]

# current_ld_path = os.environ.get('LD_LIBRARY_PATH', '')
# new_ld_path = ':'.join(intel_paths + [current_ld_path])
# os.environ['LD_LIBRARY_PATH'] = new_ld_path


# intel_libs = [
#     "/software/local/intel/parallel_studio_xe_2020_upd4/compilers_and_libraries_2020.4.304/linux/compiler/lib/intel64/libimf.so",
#     "/software/local/intel/parallel_studio_xe_2020_upd4/compilers_and_libraries_2020.4.304/linux/compiler/lib/intel64/libintlc.so.5",
#     "/software/local/intel/parallel_studio_xe_2020_upd4/compilers_and_libraries_2020.4.304/linux/compiler/lib/intel64/libsvml.so",
# ]

# for lib_path in intel_libs:
#     try:
#         ctypes.CDLL(lib_path, mode=ctypes.RTLD_GLOBAL)
#         print(f"Successfully preloaded: {lib_path}")
#     except Exception as e:
#         print(f"Failed to preload {lib_path}: {e}")

# sys.path.insert(0, '/groups/icecube/jackp/prometheus_genie_cleaned/harvard-prometheus')
# sys.path.insert(0, '/groups/icecube/jackp/prometheus_genie_cleaned/harvard-prometheus/genie_examples')

# try:
#     import h5py
#     print("✓ h5py imported successfully")
# except Exception as e:
#     print(f"✗ h5py import failed: {e}")

# try:
#     from prometheus import Prometheus, config
#     print("✓ prometheus imported successfully!")
# except Exception as e:
#     print(f"✗ prometheus import failed: {e}")
#     import traceback
#     traceback.print_exc()

Successfully preloaded: /software/local/intel/parallel_studio_xe_2020_upd4/compilers_and_libraries_2020.4.304/linux/compiler/lib/intel64/libimf.so
Successfully preloaded: /software/local/intel/parallel_studio_xe_2020_upd4/compilers_and_libraries_2020.4.304/linux/compiler/lib/intel64/libintlc.so.5
Successfully preloaded: /software/local/intel/parallel_studio_xe_2020_upd4/compilers_and_libraries_2020.4.304/linux/compiler/lib/intel64/libsvml.so
✓ h5py imported successfully
✓ prometheus imported successfully!


In [4]:

from prometheus import Prometheus, config

ModuleNotFoundError: No module named 'prometheus'

In [5]:
# import os
# os.chdir('/groups/icecube/jackp/prometheus_genie_cleaned/harvard-prometheus/genie_examples')

In [6]:
import pandas as pd
import numpy as np
import time
import sys
import argparse
import os
import logging
from datetime import datetime

sys.path.append('../')
from prometheus import Prometheus, config
import prometheus
import gc

# Import our utility functions
from genie_parser_injection import parse_and_convert_genie

from inject_in_cylinder import inject_particles_in_cylinder
from rotate_particles import rotate_particles_final

# Set up logging
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)
RESOURCE_DIR = f"{'/'.join(prometheus.__path__[0].split('/')[:-1])}/resources/"
OUTPUT_DIR = f"{'/'.join(prometheus.__path__[0].split('/')[:-1])}/genie_examples/output"


In [7]:

def main(simset, root_file_path):
    start_time = time.time()
    
    
    print(f"Using simset: {simset}")
    print(f"Using root file: {root_file_path}")
    
    # Timing for file processing and conversion in one step
    processing_start_time = time.time()
    prometheus_set, primary_set = parse_and_convert_genie(root_file_path)
    processing_end_time = time.time()
    print(f"File processing and conversion completed in {processing_end_time - processing_start_time:.2f} seconds")

    # absolute path:
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
   # output_dir = f"/groups/icecube/jackp/prometheus_genie_cleaned/harvard-prometheus/examples/output"
    primrary_file_path = f'{OUTPUT_DIR}/genie_events_primary_new_test.parquet'
    prometheus_file_path = f'{OUTPUT_DIR}/genie_events_prometheus_new_test.parquet'

    # inject into cylinder:
    primary_set, prometheus_set = inject_particles_in_cylinder(primary_set, ## neutrino information
                        prometheus_set, ## child particle information
                        cylinder_radius=500, ## meters
                        cylinder_height = 1000, # meters
                        cylinder_center = (0, 0, 0), # meters not including offset
                        )

    primary_set, prometheus_set = rotate_particles_final(primary_set, prometheus_set)


    # Timing for serialization and saving
    save_start_time = time.time()
    # position arrays to serialized strings
    prometheus_set['position'] = prometheus_set['position'].apply(lambda x: [arr.tolist() for arr in x])
    num_events = len(prometheus_set)
    print(f"Processing {num_events} events")
    prometheus_set.to_parquet(prometheus_file_path)
    primary_set.to_parquet(primrary_file_path)
    save_end_time = time.time()
    print(f"Serialization and saving completed in {save_end_time - save_start_time:.2f} seconds")

    ## GENIE stuff:
    config["injection"]["name"] = "GENIE"
    config["run"]["outfile"] = f"{OUTPUT_DIR}/100_events_{timestamp}.parquet"
    config["run"]["nevents"] = num_events
    config["injection"]["GENIE"] = config["injection"].get("GENIE", {})
    config["injection"]["GENIE"]["paths"] = config["injection"]["GENIE"].get("paths", {})
    config["injection"]["GENIE"]["inject"] = False ## we aren't injecting using prometheus, we are using an injection file
    config["injection"]["GENIE"]["simulation"] = {}
    ## geofile:
    config["detector"]["geo file"] = f"{RESOURCE_DIR}/geofiles/icecube.geo"
    ## ppc configuration:
    config['photon propagator']['name'] = 'PPC_UPGRADE'
    config["photon propagator"]["PPC_UPGRADE"]["paths"]["ppc_tmpdir"] = "./ppc_tmpdir"+str(simset)
    config["photon propagator"]["PPC_UPGRADE"]["paths"]["ppc_tmpfile"] = "ppc_tmp"+str(simset)
    config["photon propagator"]["PPC_UPGRADE"]["simulation"]["supress_output"] = False ## for printing!

    # Timing for Prometheus simulation
    sim_start_time = time.time()
    p = Prometheus(config, primary_set_parquet_path=primrary_file_path, prometheus_set_parquet_path=prometheus_file_path)
    p.sim()
    sim_end_time = time.time()
    print(f"Prometheus simulation completed in {sim_end_time - sim_start_time:.2f} seconds")
    
    # End timing
    end_time = time.time()
    total_time = end_time - start_time
    print(f"Total execution time: {total_time:.2f} seconds ({total_time/60:.2f} minutes)")
    print('Finished without catastrophic error')
    return



In [8]:
simset = 49
root_file_path = '/groups/icecube/jackp/genie_test_outputs/output_gheps/gntp_icecube_numu_100.gtac.root'

print("--------------------------------------------------------------")
print("--------------------------------------------------------------")
print("Launching simulation")
main(simset, root_file_path)
print("Finished call")
print("--------------------------------------------------------------")
print("--------------------------------------------------------------")

2025-05-14 01:23:20,028 - fsspec.local - DEBUG - open file: /groups/icecube/jackp/genie_test_outputs/output_gheps/gntp_icecube_numu_100.gtac.root
2025-05-14 01:23:20,029 - fsspec.local - DEBUG - open file: /groups/icecube/jackp/genie_test_outputs/output_gheps/gntp_icecube_numu_100.gtac.root
2025-05-14 01:23:20,114 - fsspec.local - DEBUG - open file: /groups/icecube/jackp/genie_test_outputs/output_gheps/gntp_icecube_numu_100.gtac.root
2025-05-14 01:23:20,115 - fsspec.local - DEBUG - open file: /groups/icecube/jackp/genie_test_outputs/output_gheps/gntp_icecube_numu_100.gtac.root
2025-05-14 01:23:20,126 - asyncio - DEBUG - Using selector: EpollSelector
2025-05-14 01:23:20,127 - fsspec.local - DEBUG - open file: /groups/icecube/jackp/genie_test_outputs/output_gheps/gntp_icecube_numu_100.gtac.root
2025-05-14 01:23:20,129 - fsspec.local - DEBUG - open file: /groups/icecube/jackp/genie_test_outputs/output_gheps/gntp_icecube_numu_100.gtac.root
2025-05-14 01:23:20,129 - fsspec.local - DEBUG - o

--------------------------------------------------------------
--------------------------------------------------------------
Launching simulation
Using simset: 49
Using root file: /groups/icecube/jackp/genie_test_outputs/output_gheps/gntp_icecube_numu_100.gtac.root
File processing and conversion completed in 0.16 seconds
Processing 100 events


/groups/icecube/jackp/prometheus_genie_cleaned/harvard-prometheus/genie_examples/genie_parser_injection.py:20: RuntimeWarning: invalid value encountered in scalar divide
  angle = np.arccos(np.clip(np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2)), -1, 1))


Serialization and saving completed in 0.02 seconds
using injector {'name': 'GENIE', 'GENIE': {'inject': False, 'paths': {'injection file': None}}}
photon prop:  PPCUPGRADE
position: [ -94.62225493   68.76843479 -288.18822695]
detector offset [    5.87082946    -2.51860853 -1971.9757655 ]
relative_position,  [  -88.75142547    66.24982626 -2260.16399245]
position: [-417.16738944  181.01465248 -318.82038482]
detector offset [    5.87082946    -2.51860853 -1971.9757655 ]
relative_position,  [ -411.29655998   178.49604395 -2290.79615033]
position: [-100.78577      98.12506572  199.78251486]
detector offset [    5.87082946    -2.51860853 -1971.9757655 ]
relative_position,  [  -94.91494054    95.60645719 -1772.19325064]
position: [445.73788817 151.40494249 239.27928104]
detector offset [    5.87082946    -2.51860853 -1971.9757655 ]
relative_position,  [  451.60871763   148.88633396 -1732.69648446]
position: [308.29470156 197.50453654 177.20606977]
detector offset [    5.87082946    -2.518608

Propagating MuMinus:   0%|          | 0/100 [00:00<?, ?it/s]

...

Handling charged pion/kaon/electron 211
propagating  13


Configuring ppc in "./ppc_tmpdir49/"
Configuring nextgen sensors in "./ppc_tmpdir49/"
Configuring icemodel in "./ppc_tmpdir49/"
Configuring tiltmodel in "./ppc_tmpdir49/"
Configuring holeice from "./ppc_tmpdir49/as.dat"
Configured: xR=1 eff=1.015 sf=0.137 g=0.9
Ice anisotropy is k(130)=1,1,1
New Ice anisotropy is 1.82212,0.740818,0.740818
Renorm. NI anisotropy 1.69545,0.689318,0.689318
Ice absorption anisotropy scaling is 0
Initialized BFR diffusion patterns; s_eff=0.00246854 m^-1
Loaded 12 angsens coefficients
Loaded 131072 random multipliers
Loaded 5083 cable positions
Loaded 399 RDE coefficients
 Found 4761 OMs of type -1,0
 Found 399 OMs of type -1,1
Loaded 5160 DOMs (21x19)
Loaded 80x125 2d dust layer points
Ice layer thickness: UNIFORM
Loaded 42 wavelenth points
Loaded 42 RDE coefficients
Calculated value of ppm=2268.22
Loaded 171 ice layers
Air bubble parameters: 11.7 1350 1400

Processing f2k muons from stdin on device 0
HQUO(photons/max number of hits)=1
photons: 17131  hits: 

Handling charged pion/kaon/electron 211


Propagating PiPlus:  13%|█▎        | 13/100 [00:21<02:25,  1.68s/it]


KeyboardInterrupt: 

In [7]:

primary_set_path = f'{OUTPUT_DIR}/genie_events_primary_new_test.parquet'
prometheus_set_path = f'{OUTPUT_DIR}/genie_events_prometheus_new_test.parquet'
out_parquet =  f"{OUTPUT_DIR}/100_events_20250507_140038.parquet"

prometheus_set = pd.read_parquet(prometheus_set_path)
primary_set = pd.read_parquet(primary_set_path)
out_parquet = pd.read_parquet(out_parquet)

In [9]:
primary_set

,primary,e,pdg_code,interaction,theta,phi,bjorken_x,bjorken_y,pos_x,pos_y,pos_z,position,column_depth,custom_info
0,True,28.208832,14,CC,1.532050,3.961187,-1,-1,334.183973,288.123812,41.332512,"[334.1839733527102, 288.1238115201751, 41.3325...",-1,nu:14;tgt:1000080160;N:2212;q:1(v);proc:Weak[C...
1,True,47.457617,14,NC,1.541656,6.179734,-1,-1,-108.665123,366.460294,-67.824845,"[-108.66512295503492, 366.46029387477006, -67....",-1,nu:14;tgt:1000080160;N:2112;q:2(s);proc:Weak[N...
2,True,35.966077,14,NC,1.374804,0.835516,-1,-1,-154.410123,-357.461665,-369.755297,"[-154.41012269190597, -357.4616645798619, -369...",-1,nu:14;tgt:1000080160;N:2212;q:2(v);proc:Weak[N...
3,True,13.692480,14,CC,0.776871,5.385524,-1,-1,262.529712,-381.824862,-6.161552,"[262.52971211744443, -381.82486167211925, -6.1...",-1,nu:14;tgt:1000080160;N:2112;q:1(v);proc:Weak[C...
4,True,14.574331,14,NC,1.456891,4.943279,-1,-1,-46.702952,-417.137705,-26.788787,"[-46.702951588454226, -417.13770545704614, -26...",-1,nu:14;tgt:1000080160;N:2212;q:1(v);proc:Weak[N...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,True,25.470777,14,CC,0.621649,3.966270,-1,-1,-205.447119,-146.107318,287.806482,"[-205.44711947281021, -146.10731839486178, 287...",-1,nu:14;tgt:1000010010;N:2212;q:1(v);proc:Weak[C...
96,True,37.161718,14,CC,1.317990,1.178345,-1,-1,243.050522,273.628296,-137.656420,"[243.05052220862595, 273.6282959847916, -137.6...",-1,nu:14;tgt:1000080160;N:2112;q:1(v);proc:Weak[C...
97,True,11.694786,14,CC,1.642074,4.740850,-1,-1,160.122588,-16.768930,-496.837373,"[160.12258845090668, -16.768930480940636, -496...",-1,nu:14;tgt:1000080160;N:2112;q:1(v);proc:Weak[C...
98,True,41.998553,14,CC,1.422781,2.614435,-1,-1,480.347007,98.889095,199.977681,"[480.3470071698972, 98.88909485657624, 199.977...",-1,nu:14;tgt:1000080160;N:2212;q:1(v);proc:Weak[C...


In [8]:
prometheus_set

,primary,e,pdg_code,interaction,theta,phi,bjorken_x,bjorken_y,pos_x,pos_y,pos_z,position,column_depth,custom_info
0,"[False, False, False, False, False, False, Fal...","[0.00632, 14.798849387540434, 3.46828601337033...","[22, 13, 211, 111, -211, 2212, 2212, 2212, 111]","[CC, CC, CC, CC, CC, CC, CC, CC, CC]","[2.264527079483894, 1.4693016358110564, 1.5310...","[2.339438896323253, -2.2790836196599957, -2.32...","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[334.1839733527102, 334.1839733527102, 334.183...","[288.1238115201751, 288.1238115201751, 288.123...","[41.332511570891484, 41.332511570891484, 41.33...","[[334.1839733527102, 288.1238115201751, 41.332...","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[child, child, child, child, child, child, chi..."
1,"[False, False, False, False, False, False, Fal...","[0.00618, 6.782322848774551, 12.0686641286913,...","[22, 14, 211, 211, -211, -211, 111, 2112, 111]","[NC, NC, NC, NC, NC, NC, NC, NC, NC]","[0.5747895281708876, 1.599824017476137, 1.5300...","[1.5133752804093303, 0.009790705190293507, -0....","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[-108.66512295503492, -108.66512295503492, -10...","[366.46029387477006, 366.46029387477006, 366.4...","[-67.82484450012294, -67.82484450012294, -67.8...","[[-108.66512295503492, 366.46029387477006, -67...","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[child, child, child, child, child, child, chi..."
2,"[False, False, False, False, False, False, Fal...","[0.00492, 25.00881141950755, 0.713740319020768...","[22, 14, 111, 2112, 211, 2212, 211, -211, -211...","[NC, NC, NC, NC, NC, NC, NC, NC, NC, NC]","[1.4243598401055018, 1.4285154025339162, 1.639...","[1.1320295300772318, 0.8249831850223956, 0.592...","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[-154.41012269190597, -154.41012269190597, -15...","[-357.4616645798619, -357.4616645798619, -357....","[-369.7552968096973, -369.7552968096973, -369....","[[-154.41012269190597, -357.4616645798619, -36...","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[child, child, child, child, child, child, chi..."
3,"[False, False, False, False, False, False, False]","[0.00618, 6.30803010224465, 0.1620126799194040...","[22, 13, 211, 111, 2112, 111, 2212]","[CC, CC, CC, CC, CC, CC, CC]","[0.6514766591347929, 0.8545922710179197, 0.238...","[-0.20315574701893074, -0.7040704038147179, 1....","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0]","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0]","[262.52971211744443, 262.52971211744443, 262.5...","[-381.82486167211925, -381.82486167211925, -38...","[-6.161551636789113, -6.161551636789113, -6.16...","[[262.52971211744443, -381.82486167211925, -6....","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0]","[child, child, child, child, child, child, child]"
4,"[False, False, False, False, False, False, Fal...","[12.371116111765025, 1.1125992262194506, 1.007...","[14, 2212, 2212, 2212, 2212, 2112, 2112, -211,...","[NC, NC, NC, NC, NC, NC, NC, NC, NC, NC]","[1.4215507498259923, 1.28958877508038, 1.50195...","[-1.3552028347889258, -1.4208209969408359, 1.8...","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[-46.702951588454226, -46.702951588454226, -46...","[-417.13770545704614, -417.13770545704614, -41...","[-26.788786504851487, -26.788786504851487, -26...","[[-46.702951588454226, -417.13770545704614, -2...","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[child, child, child, child, child, child, chi..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,"[False, False, False, False, False]","[13.96917571657514, 1.4131881536296862, 3.0932...","[13, 321, 321, 3122, 111]","[CC, CC, CC, CC, CC]","[0.7236938347053125, 0.5455106782215987, 0.387...","[-2.5142666961794244, -2.386848627317138, -1.2...","[-1.0, -1.0, -1.0, -1.0, -1.0]",

In [10]:
out_parquet

,mc_truth,photons
0,"{'interaction': 1, 'initial_state_energy': 28....","{'sensor_pos_x': [382.35], 'sensor_pos_y': [23..."
1,"{'interaction': 2, 'initial_state_energy': 47....","{'sensor_pos_x': [], 'sensor_pos_y': [], 'sens..."
2,"{'interaction': 2, 'initial_state_energy': 35....","{'sensor_pos_x': [-88.05], 'sensor_pos_y': [-3..."
3,"{'interaction': 1, 'initial_state_energy': 13....","{'sensor_pos_x': [], 'sensor_pos_y': [], 'sens..."
4,"{'interaction': 2, 'initial_state_energy': 14....","{'sensor_pos_x': [], 'sensor_pos_y': [], 'sens..."
...,...,...
95,"{'interaction': 1, 'initial_state_energy': 25....","{'sensor_pos_x': [-245.65], 'sensor_pos_y': [-..."
96,"{'interaction': 1, 'initial_state_energy': 37....","{'sensor_pos_x': [303.41, 330.03], 'sensor_pos..."
97,"{'interaction': 1, 'initial_state_energy': 11....","{'sensor_pos_x': [113.19, 113.19], 'sensor_pos..."
98,"{'interaction': 1, 'initial_state_energy': 41....","{'sensor_pos_x': [472.05, 303.41, 472.05, 472...."


In [12]:
out_parquet['mc_truth'][0]

{'interaction': 1,
 'initial_state_energy': 28.208831910625413,
 'initial_state_type': 14,
 'initial_state_zenith': 1.5320501683556444,
 'initial_state_azimuth': -2.3219987720974697,
 'initial_state_x': 340.0548028100745,
 'initial_state_y': 285.6052029930402,
 'initial_state_z': -1930.6432539329912,
 'final_state_energy': array([6.32000000e-03, 1.47988494e+01, 1.42527101e-02, 4.12656699e-02,
        5.01399945e-02, 3.46828601e+00, 6.62856874e+00, 1.15339384e+00,
        1.43320966e+00, 1.01739369e+00, 1.08242117e+00, 1.43302541e+00]),
 'final_state_type': array([  22.,   13.,   11.,   14.,  -12.,  211.,  111., -211., 2212.,
        2212., 2212.,  111.]),
 'final_state_zenith': array([2.26452708, 1.46930164, 1.38551193, 2.13226019, 1.17465952,
        1.53107483, 1.638726  , 1.2374756 , 1.4858955 , 1.97765853,
        0.36593032, 1.49945888]),
 'final_state_azimuth': array([ 2.3394389 , -2.27908362,  1.47899129,  2.21695426, -1.12969056,
        -2.3227409 , -2.43898324, -2.23590586, -

In [13]:
out_parquet['photons'][0]

{'sensor_pos_x': array([382.35]),
 'sensor_pos_y': array([238.9]),
 'sensor_pos_z': array([-1909.09]),
 'string_id': array([58]),
 'sensor_id': array([28]),
 't': array([613.075439]),
 'id_idx': array([9])}